# LDA - RF Pipeline 

## Import necessary packages

In [ ]:
import time
import optuna
import numpy as np
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import NMF
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
# from bitermplus import BTM

from dataset.dataset import Dataset
from constants import CLEANED_DATASET_PATH

## Constants

In [ ]:
RANDOM_SEED = 0

## Load the Hate Speech Filipino dataset from Hugging Face

In [ ]:
dataset = Dataset(full_data_path=CLEANED_DATASET_PATH,
                  from_scratch=False,
                  split_sizes=[10000, 4232, 4232])
dataset.build()

X_train = dataset.get_features(split_type="train")
Y_train = dataset.get_labels(split_type="train")
X_val = dataset.get_features(split_type="val")
Y_val = dataset.get_labels(split_type="val")
X_test = dataset.get_features(split_type="test")
Y_test = dataset.get_labels(split_type="test")

In [ ]:
# Re-split the dataset into training, validation, and test sets
# X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.2, random_state=RANDOM_SEED)
# X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=RANDOM_SEED)

## Classifier Only Pipeline

In [ ]:
def create_objective(X_train, Y_train, X_test, Y_test, random_seed):
    def objective(trial):
        # Suggest values for the hyperparameters
        n_estimators = trial.suggest_int("n_estimators", 100, 1000)

        # Create the pipeline within the trial
        pipeline = Pipeline([
            ("vectorizer", CountVectorizer(max_df=0.95, min_df=2)),
            ("rf", RandomForestClassifier(n_estimators=n_estimators, random_state=random_seed))
        ])

        # Fit the pipeline
        pipeline.fit(X_train, Y_train)

        # Evaluate the pipeline
        Y_pred = pipeline.predict(X_test)
        accuracy = accuracy_score(Y_test, Y_pred)
        return accuracy

    return objective

In [ ]:
# Create the objective function
objective = create_objective(X_train, Y_train, X_val, Y_val, RANDOM_SEED)

# Create an Optuna study object
study = optuna.create_study(direction="maximize", study_name=f"RF_Pipeline")

# Execute an optimization by running trials
study.optimize(objective, n_trials=50, n_jobs=-1)

best_rf_params = study.best_trial.params

# Best trial result
print(f"Best training accuracy: {study.best_trial.value}")
print(f"Best training hyperparameters: {best_rf_params}")

In [ ]:
# Create and fit the pipeline
pipeline = Pipeline([
    ("vectorizer", CountVectorizer(max_df=0.95, min_df=2)),
    ("rf", RandomForestClassifier(n_estimators=best_rf_params["n_estimators"], 
                                  random_state=RANDOM_SEED))
])

# Fit the pipeline
pipeline.fit(X_train, Y_train)

# Evaluate the pipeline
Y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred)

# Best testing result
print(f"Best test accuracy: {study.best_trial.value}")
print(f"Best test hyperparameters: {study.best_trial.params}")

## NMF-RF Pipeline

In [ ]:
def create_objective(X_train, Y_train, X_test, Y_test, best_rf_params, random_seed):
    def objective(trial):
        # Suggest values for the hyperparameters
        n_topics = trial.suggest_int("n_topics", 50, 300)

        # Create the pipeline within the trial
        pipeline = Pipeline([
            ("vectorizer", CountVectorizer(max_df=0.95, min_df=2)),
            ("nmf", NMF(n_components=n_topics, random_state=random_seed)),
            ("rf", RandomForestClassifier(n_estimators=best_rf_params["n_estimators"], 
                                          random_state=random_seed))
        ])

        # Fit the pipeline
        pipeline.fit(X_train, Y_train)

        # Evaluate the pipeline
        Y_pred = pipeline.predict(X_test)
        accuracy = accuracy_score(Y_test, Y_pred)
        return accuracy

    return objective

In [ ]:
# Create the objective function
objective = create_objective(X_train, Y_train, X_val, Y_val, best_rf_params, RANDOM_SEED)

# Create an Optuna study object
study = optuna.create_study(direction="maximize", study_name=f"RF_Pipeline")

# Execute an optimization by running trials
study.optimize(objective, n_trials=50, n_jobs=-1)

best_nmf_params = study.best_trial.params

# Best trial result
print(f"Best training accuracy: {study.best_trial.value}")
print(f"Best training hyperparameters: {best_nmf_params}")

In [ ]:
# Create and fit the pipeline
pipeline = Pipeline([
    ("vectorizer", CountVectorizer(max_df=0.95, min_df=2)),
    ("nmf", NMF(n_components=best_nmf_params["n_topics"], 
                random_state=RANDOM_SEED)),
    ("rf", RandomForestClassifier(n_estimators=best_rf_params["n_estimators"], 
                                  random_state=RANDOM_SEED))
])

# Fit the pipeline
pipeline.fit(X_train, Y_train)

# Evaluate the pipeline
Y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred)

# Best testing result
print(f"Best test accuracy: {study.best_trial.value}")
print(f"Best test hyperparameters: {study.best_trial.params}")